In [ ]:
import pandas as pd
import numpy as np 
from datetime import datetime
import math
import json
import os

### Reading Files

In [ ]:
intake_orig = pd.read_csv('qc-responses-1nP-questionnaire-qes_intake.csv')
gad_orig = pd.read_csv('qc-responses-1nP-questionnaire-qes_gad7.csv')
panas_orig = pd.read_csv('qc-responses-1nP-questionnaire-qes_panas10.csv')
phq_orig = pd.read_csv('qc-responses-1nP-questionnaire-qes_phq9.csv')
tapping_orig = pd.read_csv('qc-responses-1nP-task-at_tapping.csv')
stroop_orig = pd.read_csv('qc-responses-1nP-task-at_stroopeffect.csv')
health_df = pd.read_csv('qc-responses-1nP-healthdata-all.csv')

### Defining the function converting the PPT_ID to PPT_NR and saving it as json

In [ ]:
dict_path = "ppt_id_to_ppt_nr.json"

def save_dict_to_json(filename, dictionary):
    f = open(filename,"w")
    f.write(json.dumps(dictionary))
    f.close()

def load_dict_from_json(filename):
    with open(filename,"r") as f:
        return json.load(f)

def id_to_number(df):
    if os.path.exists(dict_path):
        ppt_id_to_ppt_nr = load_dict_from_json(dict_path)
    else:
        ppt_id_to_ppt_nr = {}

    for i, row in df.iterrows():
        if row['participant'] in ppt_id_to_ppt_nr:
            ppt_nr = ppt_id_to_ppt_nr[row['participant']]
        else:
            if len(ppt_id_to_ppt_nr.values()) == 0:
                ppt_nr = 1
            else:
                ppt_nr = max(ppt_id_to_ppt_nr.values()) + 1
            ppt_id_to_ppt_nr[row['participant']] = ppt_nr
        df.at[i, 'Participant_NR'] = ppt_nr
    save_dict_to_json(dict_path, ppt_id_to_ppt_nr)


### Defining the function to clean the data (dropping, renaming, and rearranging columns) 

In [ ]:
def clean_data(df):
    df_clean = df.copy()

    columns_to_drop = ['id','response_type', 'study',  'activity', 'received_at', 'time_start', 'time_end',
                        'time_scheduled_start', 'time_scheduled_end']

    df_clean.drop(columns_to_drop, axis=1, inplace=True)

    id_to_number(df_clean)

    df_clean.rename({'submission_index': 'Session', 'participant':'Participant_ID'}, axis=1, inplace=True)

    cols = df_clean.columns.tolist()
    cols = [cols[-1]] + [cols[-2]] + [cols[0]] + [cols[2]] + [cols[1]] + [cols[4]] + [cols[3]] + cols[5:-2]
    df_clean = df_clean[cols]

    df_clean.sort_values(by=['Participant_NR','Session'], inplace=True)

    return df_clean

    

### Calling the clean data function on the intake, mood questionnaires and tapping task

In [ ]:
intake_clean = clean_data(intake_orig)
gad_clean = clean_data(gad_orig)
phq_clean = clean_data(phq_orig)
panas_clean = clean_data(panas_orig)
tapping_clean = clean_data(tapping_orig)

### Clean Intake Data

In [ ]:
intake_clean

,Participant_NR,Session,Participant_ID,app_version,study_version,submitted_at,timezone,Date_as_Number,Gender,Age,...,prescription_meds,blood_circulation_problems,blood_circulation_type,heart_vascular_disorders,heart_vascular_type,musculoskeletal_concerns,musculoskeletal_type,respiratory_concerns,respiratory_type,symptoms_list
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (49),4,2021-02-22T08:53:28.106930,CET,20210222,Male,55-64,...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1,b0354b51-fedf-4445-b2e2-335141af6709,1.0.0 - QC Pro (50),24,2021-02-23T10:43:10.918661,CET,20210223,Female,55-64,...,Yes,No,NaN,No,NaN,No,NaN,No,NaN,Other
2,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-23T23:02:51.756974,CET,20210223,Female,55-64,...,Yes,No,NaN,Yes,Atherosclerosis,Yes,Other,Yes,Chronic Obstructive Pulmonary Disease,Shortness of Breath
3,4.0,1,d888256d-5657-4813-a057-95d57ec9716d,1.0.0 - QC Pro (54),24,2021-02-24T07:33:24.119574,CET,20210224,Male,35-44,...,Yes,No,NaN,No,NaN,No,NaN,No,NaN,Other
4,5.0,1,a6e378f3-faac-470d-a7d4-4de6727b2f4c,1.0.0 - QC Pro (54),24,2021-02-24T08:45:42.553438,CET,20210224,Female,35-44,...,Yes,No,NaN,No,NaN,No,NaN,No,NaN,Headaches
5,6.0,1,117201e3-bcd7-4d5e-b472-39ac9a6ac717,1.0.0 - QC Pro (54),24,2021-02-24T09:06:47.392363,CET,20210224,Female,25-34,...,No,No,NaN,No,NaN,No,NaN,No,NaN,Weight Gain
6,7.0,1,687d06d2-23b7-4bb1-ac97-f30c08d5b315,1.0.0 - QC Pro (54),24,2021-02-24T10:17:44.200668,CET,20210224,Female,25-34,...,No,No,NaN,No,NaN,No,NaN,No,NaN,Other
8,8.0,1,7bb1ebbd-a166-49de-aac7-53037be8d595,1.0.0 - QC Pro (54),24,2021-02-25T11:38:09.811575,CET,20210225,Male,55-64,...,Yes,Yes,Other,No,NaN,No,NaN,No,NaN,Insomnia
7,9.0,1,622a085e-6b7a-47c6-ae4e-6894dd81343e,1.0.0 - QC Pro (54),24,2021-02-24T16:00:41.062274,CET,20210224,Non-Binary,25-34,...,No,No,NaN,No,NaN,No,NaN,No,NaN,Low Mood
9,11.0,1,75ff5520-469d-478b-b30b-bfa182e14bf9,1.0.0 - QC Pro (54),25,2021-03-01T13:28:01.548708,CET,20210301,Female,35-44,...,No,No,NaN,No,NaN,No,NaN,No,NaN,Low Mood


### Clean Tapping Task Data

In [ ]:
tapping_clean

,Participant_NR,Session,Participant_ID,app_version,study_version,submitted_at,timezone,Date_as_Number,Correct_Right_Hand,Correct_Left_Hand,Incorrect_Right_Hand,Incorrect_Left_Hand,Missing_data
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (49),21,2021-02-22T17:59:04.734536,CET,20210222,153,153,2,2,0
8,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-01T19:22:28.348200,CET,20210301,127,126,24,19,0
16,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (58),25,2021-03-10T10:02:28.618124,CET,20210310,131,132,21,31,0
23,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-17T09:01:22.601142,CET,20210317,155,155,5,5,0
32,1.0,5,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T20:03:18.490087,CET,20210322,97,96,24,20,0
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:53:41.765440,CET,20210224,116,116,0,4,0
9,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:26:59.115100,CET,20210302,79,78,5,42,0
15,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:29:28.214104,CET,20210309,124,123,2,13,0
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:25:47.903546,CET,20210316,114,114,1,20,0
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:42:53.182447,CET,20210323,120,120,1,5,0


In [ ]:
tapping_clean.submitted_at.max()

'2021-04-09T19:57:44.349485'

### GAD_7: scoring the data

In [ ]:
gad_clean.columns.to_list()

['Participant_NR',
 'Session',
 'Participant_ID',
 'app_version',
 'study_version',
 'submitted_at',
 'timezone',
 'Date_as_Number',
 'question1',
 'question9',
 'question8',
 'question7',
 'question6',
 'question5',
 'question4']

In [ ]:
gad_col = ['question1', 'question9', 'question8', 'question7', 'question6', 'question5', 'question4']

for i, col in enumerate(gad_col):
    to_rename = {}
    to_rename[col] = f'GAD7_{i+1}'
    gad_clean.rename(to_rename, axis=1, inplace=True)


encoding = {
    'Not at all': 0, 
    'Several days': 1, 
    'More than half the days': 2, 
    'Nearly every day': 3
}


for i in range(7):
    old_col = f'GAD7_{i+1}'
    new_col = f'GAD7_{i+1}_Encoded'
    for j, row in gad_clean.iterrows():
        gad_clean.at[j, new_col] = encoding[row[old_col]]

gad_clean['GAD7_Total'] = 0

for i, row in gad_clean.iterrows():
    cell = row[gad_clean.columns[15:]].sum()
    gad_clean.at[i, 'GAD7_Total'] = cell

gad_clean

,Participant_NR,Session,Participant_ID,app_version,study_version,submitted_at,timezone,Date_as_Number,GAD7_1,GAD7_2,...,GAD7_6,GAD7_7,GAD7_1_Encoded,GAD7_2_Encoded,GAD7_3_Encoded,GAD7_4_Encoded,GAD7_5_Encoded,GAD7_6_Encoded,GAD7_7_Encoded,GAD7_Total
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (49),21,2021-02-22T17:57:57.785120,CET,20210222,Not at all,Not at all,...,Not at all,Not at all,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2
8,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-01T19:21:41.089886,CET,20210301,Not at all,Not at all,...,Not at all,Not at all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (58),25,2021-03-10T10:01:40.240269,CET,20210310,Not at all,Not at all,...,Not at all,Not at all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
23,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-17T09:00:40.480148,CET,20210317,Not at all,Not at all,...,Not at all,Not at all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
32,1.0,5,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T15:12:45.832603,CET,20210322,Not at all,Not at all,...,Not at all,Not at all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:51:10.536009,CET,20210224,Not at all,Several days,...,Several days,Not at all,0.0,1.0,1.0,1.0,1.0,1.0,0.0,5
9,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:25:02.442685,CET,20210302,Several days,Not at all,...,Several days,Not at all,1.0,0.0,0.0,1.0,1.0,1.0,0.0,4
15,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:27:47.454130,CET,20210309,Not at all,Not at all,...,Several days,Not at all,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:23:11.900292,CET,20210316,Several days,Several days,...,Several days,Several days,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:44:27.084087,CET,20210323,Several days,Several days,...,Not at all,Several days,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5


In [ ]:
gad_clean.submitted_at.max()

'2021-04-09T19:56:15.794148'

### PHQ_9: scoring the data

In [ ]:
phq_clean.columns.to_list()


['Participant_NR',
 'Session',
 'Participant_ID',
 'app_version',
 'study_version',
 'submitted_at',
 'timezone',
 'Date_as_Number',
 'question1',
 'question8',
 'question7',
 'question6',
 'question5',
 'question4',
 'question3',
 'question2',
 'question9',
 'question10']

In [ ]:
phq_col = ['question1', 'question8', 'question7', 'question6', 'question5', 'question4', 'question3',
         'question2', 'question9','question10']

for i, col in enumerate(phq_col):
    to_rename = {}
    to_rename[col] = f'PHQ9_{i+1}'
    phq_clean.rename(to_rename, axis=1, inplace=True)

encoding = {
    'Not at all': 0, 
    'Several days': 1, 
    'More than half the days': 2, 
    'Nearly every day': 3
}

encoding2 = {
    'Not difficult at all': 0, 
    'Somewhat difficult': 1, 
    'Very difficult': 2, 
    'Extremely difficult': 3
}

for i in range(9):
    old_col = f'PHQ9_{i+1}'
    new_col = f'PHQ9_{i+1}_Encoded'
    for j, row in phq_clean.iterrows():
        phq_clean.at[j, new_col] = encoding[row[old_col]]

for i, row in phq_clean.iterrows():
    phq_clean.at[i, 'PHQ9_10_Encoded'] = encoding2[row['PHQ9_10']]

phq_clean['PHQ9_Total'] = 0

for i, row in phq_clean.iterrows():
    cell = row[phq_clean.columns[18:]].sum()
    phq_clean.at[i, 'PHQ9_Total'] = cell

cols = phq_clean.columns.tolist()
cols = cols[:17] + cols[18:27] + [cols[28]] + [cols[17]] + [cols[27]]
phq_clean = phq_clean[cols]

phq_clean['Suicidality'] = ''

for i, row in phq_clean.iterrows():
    
    if row['PHQ9_9_Encoded'] == 0:
        cell = False
    elif row['PHQ9_9_Encoded'] != 0: 
        cell = True
    phq_clean.at[i, 'Suicidality'] = cell

phq_clean

,Participant_NR,Session,Participant_ID,app_version,study_version,submitted_at,timezone,Date_as_Number,PHQ9_1,PHQ9_2,...,PHQ9_4_Encoded,PHQ9_5_Encoded,PHQ9_6_Encoded,PHQ9_7_Encoded,PHQ9_8_Encoded,PHQ9_9_Encoded,PHQ9_Total,PHQ9_10,PHQ9_10_Encoded,Suicidality
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (49),21,2021-02-22T17:57:06.651879,CET,20210222,Not at all,Not at all,...,1.0,0.0,0.0,0.0,0.0,0.0,1,Not difficult at all,0.0,False
8,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-01T19:20:57.373288,CET,20210301,Not at all,Not at all,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Not difficult at all,0.0,False
16,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (58),25,2021-03-10T10:01:04.622279,CET,20210310,Not at all,Not at all,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Not difficult at all,0.0,False
23,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-17T09:00:10.766668,CET,20210317,Not at all,Not at all,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Not difficult at all,0.0,False
32,1.0,5,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T15:12:20.183868,CET,20210322,Not at all,Not at all,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Not difficult at all,0.0,False
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:50:21.338992,CET,20210224,More than half the days,Not at all,...,3.0,1.0,3.0,1.0,0.0,0.0,14,Somewhat difficult,1.0,False
9,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:24:05.565211,CET,20210302,Several days,Not at all,...,2.0,1.0,3.0,2.0,0.0,0.0,13,Somewhat difficult,1.0,False
15,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:26:56.116192,CET,20210309,Several days,Not at all,...,2.0,1.0,2.0,1.0,0.0,0.0,11,Somewhat difficult,1.0,False
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:21:24.756638,CET,20210316,Several days,Several days,...,3.0,0.0,2.0,2.0,0.0,0.0,13,Somewhat difficult,1.0,False
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:41:56.428718,CET,20210323,Several days,Not at all,...,1.0,0.0,2.0,2.0,0.0,0.0,10,Somewhat difficult,1.0,False


### PANAS_10: scoring the data

In [ ]:
panas_clean.columns.to_list()

['Participant_NR',
 'Session',
 'Participant_ID',
 'app_version',
 'study_version',
 'submitted_at',
 'timezone',
 'Date_as_Number',
 'question2',
 'question1',
 'question3',
 'question4',
 'question5',
 'question6',
 'question7',
 'question8',
 'question9',
 'question10',
 'question11',
 'question12',
 'question13',
 'question14',
 'question15',
 'question16',
 'question17',
 'question18',
 'question19',
 'question20']

In [ ]:
panas_col = [ 'question2', 'question1', 'question3', 'question4', 'question5', 'question6', 'question7',
            'question8', 'question9', 'question10', 'question11', 'question12', 'question13', 'question14',
            'question15', 'question16', 'question17', 'question18', 'question19', 'question20']

for i, col in enumerate(panas_col):
    to_rename = {}
    to_rename[col] = f'PANAS10_{i+1}'
    panas_clean.rename(to_rename, axis=1, inplace=True)

encoding = {
    'Very Slightly or Not at All': 1, 
    'A Little': 2, 
    'Moderately': 3, 
    'Quite a Bit': 4,
    # TWO EXTRA SPACES after EXtremely (in json/ panas-10 )
    'Extremely  ': 5
}


for i in range(20):
    old_col = f'PANAS10_{i+1}'
    new_col = f'PANAS10_{i+1}_Encoded'
    for j, row in panas_clean.iterrows():
        panas_clean.at[j, new_col] = encoding[row[old_col]]


col_positive = ['PANAS10_1_Encoded', 'PANAS10_3_Encoded', 'PANAS10_5_Encoded', 'PANAS10_9_Encoded', 'PANAS10_10_Encoded', 
                'PANAS10_12_Encoded', 'PANAS10_14_Encoded', 'PANAS10_16_Encoded', 'PANAS10_17_Encoded', 'PANAS10_19_Encoded', ]
col_negative = ['PANAS10_2_Encoded', 'PANAS10_4_Encoded', 'PANAS10_6_Encoded', 'PANAS10_7_Encoded', 'PANAS10_8_Encoded', 
                'PANAS10_11_Encoded', 'PANAS10_13_Encoded', 'PANAS10_15_Encoded', 'PANAS10_18_Encoded', 'PANAS10_20_Encoded', ]

panas_clean['PANAS10_Positive'] = 0
panas_clean['PANAS10_Negative'] = 0

for i, row in panas_clean.iterrows():
    
    cell_positive = row[col_positive].sum()
    cell_negative = row[col_negative].sum()
    panas_clean.at[i, 'PANAS10_Positive'] = cell_positive
    panas_clean.at[i, 'PANAS10_Negative'] = cell_negative

panas_clean

### Stroop: additional cleaning

In [ ]:
date_time = []
for i in range(30):
    date_time.append(f'Inter{i+1}_Date_Time')

In [ ]:
for i, row in enumerate(stroop_orig.time_start):
    cell = datetime.fromisoformat(stroop_orig['time_start'][i]).timestamp()
    stroop_orig.at[i, 'Time_Start'] = cell

timestamps = []
for i in range(30):
    timestamps.append(f'Inter{i+1}_Timestamp')

stroop_orig[timestamps] =  ''


for i, row in stroop_orig.iterrows():
    for j in range(30):
        if type(row[f'Inter{j+1}_Date_Time']) == str:
            cell = datetime.fromisoformat(row[f'Inter{j+1}_Date_Time']).timestamp()
        else:
            cell = None
        stroop_orig.at[i, f'Inter{j+1}_Timestamp'] = cell


In [ ]:
timestamps = ['Time_Start']
for i in range(30):
    timestamps.append(f'Inter{i+1}_Timestamp')

In [ ]:
for i, row in stroop_orig.iterrows():
    for j, _ in enumerate(timestamps):
        if j == len(timestamps) - 1:
            break
        try:
            cell = row[timestamps[j + 1]] - row[timestamps[j]]
            
        except (IndexError, TypeError):
            pass
        stroop_orig.at[i, f'Inter{j+1}_Time'] = round(cell, 3)

In [ ]:
columns_to_drop2 = ['id','response_type', 'study',  'activity', 'received_at', 'time_end',
                        'time_scheduled_start', 'time_scheduled_end']

stroop_to_drop = timestamps + columns_to_drop2
stroop_clean = stroop_orig.drop(stroop_to_drop, axis=1)

id_to_number(stroop_clean)

stroop_clean.rename({'submission_index': 'Session', 'participant':'Participant_ID'}, axis=1, inplace=True)

cols = stroop_clean.columns.tolist()
cols = [cols[-1]] + [cols[127]] + [cols[0]] + [cols[2]] + [cols[1]] + [cols[4]] + [cols[3]] + cols[5:127] + cols[128:-1]
stroop_clean = stroop_clean[cols]
    
stroop_clean.sort_values(by=['Participant_NR', 'Session'], inplace=True)

stroop_clean

,Participant_NR,Session,Participant_ID,app_version,study_version,time_start,timezone,submitted_at,Date_as_Number,Inter1_Date_Time,...,Inter21_Time,Inter22_Time,Inter23_Time,Inter24_Time,Inter25_Time,Inter26_Time,Inter27_Time,Inter28_Time,Inter29_Time,Inter30_Time
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (50),24,2021-02-23T14:26:00.528202,CET,2021-02-23T14:26:32.317471,20210223,2021-02-23T14:26:02.292107,...,1.756,1.756,1.756,1.756,1.756,1.756,1.756,1.756,1.756,1.756
11,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-04T09:12:01.738684,CET,2021-03-04T09:12:32.894940,20210304,2021-03-04T09:12:03.504572,...,1.543,1.543,1.543,1.543,1.543,1.543,1.543,1.543,1.543,1.543
24,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-18T08:11:38.173258,CET,2021-03-18T08:12:09.404013,20210318,2021-03-18T08:11:39.938095,...,1.763,1.763,1.763,1.763,1.763,1.763,1.763,1.763,1.763,1.763
32,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T16:37:01.555843,CET,2021-03-22T16:37:34.186638,20210322,2021-03-22T16:37:03.313937,...,1.941,1.941,1.941,1.941,1.941,1.941,1.941,1.941,1.941,1.941
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:52:02.263546,CET,2021-02-24T15:52:35.671954,20210224,2021-02-24T15:52:04.027113,...,1.006,1.006,1.006,1.006,1.006,1.006,1.006,1.006,1.006,1.006
8,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:25:18.039207,CET,2021-03-02T15:25:50.077679,20210302,2021-03-02T15:25:19.800074,...,1.764,1.764,1.764,1.764,1.764,1.764,1.764,1.764,1.764,1.764
16,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:28:02.434281,CET,2021-03-09T12:28:38.700688,20210309,2021-03-09T12:28:04.198741,...,1.005,1.004,1.004,1.004,1.004,1.004,1.004,1.004,1.004,1.004
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:24:14.085359,CET,2021-03-16T18:24:48.340917,20210316,2021-03-16T18:24:15.847032,...,1.006,1.006,1.006,1.006,1.006,1.006,1.006,1.006,1.006,1.006
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:43:06.003108,CET,2021-03-23T12:43:38.577442,20210323,2021-03-23T12:43:07.747554,...,1.851,1.851,1.851,1.851,1.851,1.851,1.851,1.851,1.851,1.851
2,5.0,1,a6e378f3-faac-470d-a7d4-4de6727b2f4c,1.0.0 - QC Pro (54),24,2021-02-24T14:36:04.442605,CET,2021-02-24T14:36:35.653614,20210224,2021-02-24T14:36:06.208646,...,1.900,1.900,1.900,1.900,1.900,1.900,1.900,1.900,1.900,1.900


### Stroop scoring

In [ ]:
time = []
for i in range(30):
    time.append(f'Inter{i+1}_Time')

stroop_clean['Average_Speed'] = round(stroop_clean[time].mean(axis = 1), 2)

In [ ]:
correct_col = []
for i in range(30):
    correct_col.append(f'Inter{i+1}_Correct')

In [ ]:
for i, row in stroop_clean.iterrows():
    for j, col in enumerate(correct_col):
        if row[col] == 'Correct gesture':
            cell = True
        elif row[col] == 'Wrong gesture':
            cell = False
        else:
            cell = None

        stroop_clean.at[i, f'Inter{j+1}_Correct'] = cell

In [ ]:
stroop_clean['Congruent_Correct_Total'] = ''
stroop_clean['Congruent_Incorrect_Total'] = ''
stroop_clean['Incongruent_Correct_Total'] = ''
stroop_clean['Incongruent_Incorrect_Total'] = ''
stroop_clean['Not_filled'] = ''

for i, row in stroop_clean.iterrows():
    cell_congruent_correct = 0
    cell_congruent_incorrect = 0
    cell_incongruent_correct = 0
    cell_incongruent_incorrect = 0
    cell_not_filled = 0
    for j in range(30):
        correct = row[f'Inter{j+1}_Correct']
        color = row[f'Inter{j+1}_Color']
        spelling = row[f'Inter{j+1}_Spelling']
        if isinstance(correct, bool) and isinstance(color, str) and isinstance(spelling, str):
            if correct and color ==  spelling:
                cell_congruent_correct += 1
            elif not correct and color == spelling:
                cell_congruent_incorrect += 1
            elif correct and color != spelling:
                cell_incongruent_correct += 1
            elif not correct and color != spelling:
                cell_incongruent_incorrect += 1
        else:
            cell_not_filled += 1

       
        stroop_clean.at[i, 'Congruent_Correct_Total'] = cell_congruent_correct
        stroop_clean.at[i, 'Congruent_Incorrect_Total'] = cell_congruent_incorrect
        stroop_clean.at[i, 'Incongruent_Correct_Total'] = cell_incongruent_correct
        stroop_clean.at[i, 'Incongruent_Incorrect_Total'] = cell_incongruent_incorrect
        stroop_clean.at[i, 'Not_filled'] = cell_not_filled 

stroop_clean.drop('Not_filled', axis= 1, inplace= True)

In [ ]:
stroop_clean['Congruent_Correct_Perc'] = ''
stroop_clean['Congruent_Incorrect_Perc'] = ''
stroop_clean['Incongruent_Correct_Perc'] = ''
stroop_clean['Incongruent_Incorrect_Perc'] = ''

for i, row in stroop_clean.iterrows():
    congruent_correct_av = 0
    congruent_incorrect_av = 0
    incongruent_correct_av = 0
    incongruent_incorrect_av = 0
    
    for j in range(30):
        all_total = (row['Congruent_Correct_Total'] + row['Congruent_Incorrect_Total'] + row['Incongruent_Correct_Total'] + row['Incongruent_Incorrect_Total'])
        congruent_correct_av = (row['Congruent_Correct_Total'] / all_total) * 100
        congruent_incorrect_av = (row['Congruent_Incorrect_Total'] / all_total) * 100
        incongruent_correct_av = (row['Incongruent_Correct_Total'] / all_total) * 100
        incongruent_incorrect_av = (row['Incongruent_Incorrect_Total'] / all_total) * 100
       
        stroop_clean.at[i, 'Congruent_Correct_Perc'] = round(congruent_correct_av, 2)
        stroop_clean.at[i, 'Congruent_Incorrect_Perc'] = round(congruent_incorrect_av, 2)
        stroop_clean.at[i, 'Incongruent_Correct_Perc'] = round(incongruent_correct_av, 2)
        stroop_clean.at[i, 'Incongruent_Incorrect_Perc'] = round(incongruent_incorrect_av, 2)

stroop_clean

,Participant_NR,Session,Participant_ID,app_version,study_version,time_start,timezone,submitted_at,Date_as_Number,Inter1_Date_Time,...,Inter30_Time,Average_Speed,Congruent_Correct_Total,Congruent_Incorrect_Total,Incongruent_Correct_Total,Incongruent_Incorrect_Total,Congruent_Correct_Perc,Congruent_Incorrect_Perc,Incongruent_Correct_Perc,Incongruent_Incorrect_Perc
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (50),24,2021-02-23T14:26:00.528202,CET,2021-02-23T14:26:32.317471,20210223,2021-02-23T14:26:02.292107,...,1.756,1.76,1,3,0,13,5.88,17.65,0.0,76.47
11,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-04T09:12:01.738684,CET,2021-03-04T09:12:32.894940,20210304,2021-03-04T09:12:03.504572,...,1.543,1.66,1,3,0,13,5.88,17.65,0.0,76.47
24,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-18T08:11:38.173258,CET,2021-03-18T08:12:09.404013,20210318,2021-03-18T08:11:39.938095,...,1.763,1.76,0,3,0,14,0.0,17.65,0.0,82.35
32,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T16:37:01.555843,CET,2021-03-22T16:37:34.186638,20210322,2021-03-22T16:37:03.313937,...,1.941,1.83,1,1,2,13,5.88,5.88,11.76,76.47
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:52:02.263546,CET,2021-02-24T15:52:35.671954,20210224,2021-02-24T15:52:04.027113,...,1.006,1.46,1,3,0,13,5.88,17.65,0.0,76.47
8,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:25:18.039207,CET,2021-03-02T15:25:50.077679,20210302,2021-03-02T15:25:19.800074,...,1.764,1.76,3,2,1,11,17.65,11.76,5.88,64.71
16,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:28:02.434281,CET,2021-03-09T12:28:38.700688,20210309,2021-03-09T12:28:04.198741,...,1.004,1.46,0,2,3,12,0.0,11.76,17.65,70.59
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:24:14.085359,CET,2021-03-16T18:24:48.340917,20210316,2021-03-16T18:24:15.847032,...,1.006,1.46,1,4,2,10,5.88,23.53,11.76,58.82
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:43:06.003108,CET,2021-03-23T12:43:38.577442,20210323,2021-03-23T12:43:07.747554,...,1.851,1.80,5,1,6,5,29.41,5.88,35.29,29.41
2,5.0,1,a6e378f3-faac-470d-a7d4-4de6727b2f4c,1.0.0 - QC Pro (54),24,2021-02-24T14:36:04.442605,CET,2021-02-24T14:36:35.653614,20210224,2021-02-24T14:36:06.208646,...,1.900,1.82,1,1,0,15,5.88,5.88,0.0,88.24


In [ ]:
stroop_clean['Congruent_Time_Mean'] = ''
stroop_clean['Incongruent_Time_Mean'] = ''
# stroop_clean['Time_Mean_Difference'] = ''

for i, row in stroop_clean.iterrows():
    congruent_time_total = 0
    incongruent_time_total = 0
    
    for j in range(30):
        correct = row[f'Inter{j+1}_Correct']
        color = row[f'Inter{j+1}_Color']
        spelling = row[f'Inter{j+1}_Spelling']
        if isinstance(correct, bool) and isinstance(color, str) and isinstance(spelling, str):
            if correct and color ==  spelling:
                congruent_time_total += row[time[j]]
            elif correct and color != spelling:
                incongruent_time_total += row[time[j]]
            else:
                cell = 0
        if row['Congruent_Correct_Total'] > 1:
            congruent_time_mean = congruent_time_total / row['Congruent_Correct_Total']
        else:
            congruent_time_mean = congruent_time_total
        if row['Incongruent_Correct_Total'] > 1:
            incongruent_time_mean = incongruent_time_total / row['Incongruent_Correct_Total']
        else:
            incongruent_time_mean = incongruent_time_total
    
    stroop_clean.at[i, 'Congruent_Time_Mean'] = round(congruent_time_mean, 3) if congruent_time_mean != 0 else np.nan
    stroop_clean.at[i, 'Incongruent_Time_Mean'] = round(incongruent_time_mean, 3) if incongruent_time_mean != 0 else np.nan



stroop_clean.head(15)


,Participant_NR,Session,Participant_ID,app_version,study_version,time_start,timezone,submitted_at,Date_as_Number,Inter1_Date_Time,...,Congruent_Correct_Total,Congruent_Incorrect_Total,Incongruent_Correct_Total,Incongruent_Incorrect_Total,Congruent_Correct_Perc,Congruent_Incorrect_Perc,Incongruent_Correct_Perc,Incongruent_Incorrect_Perc,Congruent_Time_Mean,Incongruent_Time_Mean
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (50),24,2021-02-23T14:26:00.528202,CET,2021-02-23T14:26:32.317471,20210223,2021-02-23T14:26:02.292107,...,1,3,0,13,5.88,17.65,0.0,76.47,1.534,NaN
11,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-04T09:12:01.738684,CET,2021-03-04T09:12:32.894940,20210304,2021-03-04T09:12:03.504572,...,1,3,0,13,5.88,17.65,0.0,76.47,1.543,NaN
24,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-18T08:11:38.173258,CET,2021-03-18T08:12:09.404013,20210318,2021-03-18T08:11:39.938095,...,0,3,0,14,0.0,17.65,0.0,82.35,NaN,NaN
32,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T16:37:01.555843,CET,2021-03-22T16:37:34.186638,20210322,2021-03-22T16:37:03.313937,...,1,1,2,13,5.88,5.88,11.76,76.47,1.67,1.543
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:52:02.263546,CET,2021-02-24T15:52:35.671954,20210224,2021-02-24T15:52:04.027113,...,1,3,0,13,5.88,17.65,0.0,76.47,2.574,NaN
8,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:25:18.039207,CET,2021-03-02T15:25:50.077679,20210302,2021-03-02T15:25:19.800074,...,3,2,1,11,17.65,11.76,5.88,64.71,1.694,0.814
16,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:28:02.434281,CET,2021-03-09T12:28:38.700688,20210309,2021-03-09T12:28:04.198741,...,0,2,3,12,0.0,11.76,17.65,70.59,NaN,1.687
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:24:14.085359,CET,2021-03-16T18:24:48.340917,20210316,2021-03-16T18:24:15.847032,...,1,4,2,10,5.88,23.53,11.76,58.82,1.826,1.692
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:43:06.003108,CET,2021-03-23T12:43:38.577442,20210323,2021-03-23T12:43:07.747554,...,5,1,6,5,29.41,5.88,35.29,29.41,1.706,1.711
2,5.0,1,a6e378f3-faac-470d-a7d4-4de6727b2f4c,1.0.0 - QC Pro (54),24,2021-02-24T14:36:04.442605,CET,2021-02-24T14:36:35.653614,20210224,2021-02-24T14:36:06.208646,...,1,1,0,15,5.88,5.88,0.0,88.24,1.694,NaN


In [ ]:
for i, row in stroop_clean.iterrows():
    if row['Congruent_Time_Mean'] != 0 and row['Incongruent_Time_Mean'] != 0:
        diff_time = row['Incongruent_Time_Mean'] - row['Congruent_Time_Mean']
    elif row['Congruent_Time_Mean'] != 0 and row['Incongruent_Time_Mean'] == 0:
        diff_time = row['Incongruent_Time_Mean'] - row['Congruent_Time_Mean']
    elif row['Congruent_Time_Mean'] == 0 and row['Incongruent_Time_Mean'] != 0:
        diff_time =  row['Incongruent_Time_Mean'] - row['Congruent_Time_Mean']
    else:
        diff_time = 0
    stroop_clean.at[i, 'Mean_Time_Difference'] = round(diff_time, 3)
stroop_clean.head(10)

,Participant_NR,Session,Participant_ID,app_version,study_version,time_start,timezone,submitted_at,Date_as_Number,Inter1_Date_Time,...,Congruent_Incorrect_Total,Incongruent_Correct_Total,Incongruent_Incorrect_Total,Congruent_Correct_Perc,Congruent_Incorrect_Perc,Incongruent_Correct_Perc,Incongruent_Incorrect_Perc,Congruent_Time_Mean,Incongruent_Time_Mean,Mean_Time_Difference
0,1.0,1,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (50),24,2021-02-23T14:26:00.528202,CET,2021-02-23T14:26:32.317471,20210223,2021-02-23T14:26:02.292107,...,3,0,13,5.88,17.65,0.0,76.47,1.534,NaN,NaN
11,1.0,2,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (54),25,2021-03-04T09:12:01.738684,CET,2021-03-04T09:12:32.894940,20210304,2021-03-04T09:12:03.504572,...,3,0,13,5.88,17.65,0.0,76.47,1.543,NaN,NaN
24,1.0,3,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-18T08:11:38.173258,CET,2021-03-18T08:12:09.404013,20210318,2021-03-18T08:11:39.938095,...,3,0,14,0.0,17.65,0.0,82.35,NaN,NaN,NaN
32,1.0,4,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,1.0.0 - QC Pro (60),25,2021-03-22T16:37:01.555843,CET,2021-03-22T16:37:34.186638,20210322,2021-03-22T16:37:03.313937,...,1,2,13,5.88,5.88,11.76,76.47,1.67,1.543,-0.127
3,3.0,1,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),24,2021-02-24T15:52:02.263546,CET,2021-02-24T15:52:35.671954,20210224,2021-02-24T15:52:04.027113,...,3,0,13,5.88,17.65,0.0,76.47,2.574,NaN,NaN
8,3.0,2,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (54),25,2021-03-02T15:25:18.039207,CET,2021-03-02T15:25:50.077679,20210302,2021-03-02T15:25:19.800074,...,2,1,11,17.65,11.76,5.88,64.71,1.694,0.814,-0.880
16,3.0,3,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (58),25,2021-03-09T12:28:02.434281,CET,2021-03-09T12:28:38.700688,20210309,2021-03-09T12:28:04.198741,...,2,3,12,0.0,11.76,17.65,70.59,NaN,1.687,NaN
22,3.0,4,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (59),25,2021-03-16T18:24:14.085359,CET,2021-03-16T18:24:48.340917,20210316,2021-03-16T18:24:15.847032,...,4,2,10,5.88,23.53,11.76,58.82,1.826,1.692,-0.134
33,3.0,5,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,1.0.0 - QC Pro (60),25,2021-03-23T12:43:06.003108,CET,2021-03-23T12:43:38.577442,20210323,2021-03-23T12:43:07.747554,...,1,6,5,29.41,5.88,35.29,29.41,1.706,1.711,0.005
2,5.0,1,a6e378f3-faac-470d-a7d4-4de6727b2f4c,1.0.0 - QC Pro (54),24,2021-02-24T14:36:04.442605,CET,2021-02-24T14:36:35.653614,20210224,2021-02-24T14:36:06.208646,...,1,0,15,5.88,5.88,0.0,88.24,1.694,NaN,NaN


In [ ]:
stroop_clean.to_csv('stroop_clean_updated.22-04.csv', index=False)

### Creating Stroop Basic

In [ ]:
stroop_basic_cols = stroop_clean.columns.tolist()

In [ ]:
stroop_basic_cols = [stroop_basic_cols[0]] + [stroop_basic_cols[2]] + [stroop_basic_cols[8]] + stroop_basic_cols[159:]
stroop_basic = stroop_clean[stroop_basic_cols]

stroop_basic

,Participant_NR,Participant_ID,Date_as_Number,Average_Speed,Congruent_Correct_Total,Congruent_Incorrect_Total,Incongruent_Correct_Total,Incongruent_Incorrect_Total,Congruent_Correct_Perc,Congruent_Incorrect_Perc,Incongruent_Correct_Perc,Incongruent_Incorrect_Perc,Congruent_Time_Mean,Incongruent_Time_Mean,Mean_Time_Difference
0,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210223,1.76,1,3,0,13,5.88,17.65,0.0,76.47,1.534,NaN,NaN
11,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210304,1.66,1,3,0,13,5.88,17.65,0.0,76.47,1.543,NaN,NaN
24,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210318,1.76,0,3,0,14,0.0,17.65,0.0,82.35,NaN,NaN,NaN
32,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210322,1.83,1,1,2,13,5.88,5.88,11.76,76.47,1.67,1.543,-0.127
3,3.0,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,20210224,1.46,1,3,0,13,5.88,17.65,0.0,76.47,2.574,NaN,NaN
8,3.0,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,20210302,1.76,3,2,1,11,17.65,11.76,5.88,64.71,1.694,0.814,-0.880
16,3.0,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,20210309,1.46,0,2,3,12,0.0,11.76,17.65,70.59,NaN,1.687,NaN
22,3.0,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,20210316,1.46,1,4,2,10,5.88,23.53,11.76,58.82,1.826,1.692,-0.134
33,3.0,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,20210323,1.80,5,1,6,5,29.41,5.88,35.29,29.41,1.706,1.711,0.005
2,5.0,a6e378f3-faac-470d-a7d4-4de6727b2f4c,20210224,1.82,1,1,0,15,5.88,5.88,0.0,88.24,1.694,NaN,NaN


In [ ]:
stroop_basic.to_csv('stroop_basic.updated.22-04.csv', index=False)

## Health Data

In [ ]:
health_df.head()

,id,participant,response_type,study,study_version,activity,app_version,timezone,received_at,time_start,...,Date_as_Number,Total_Dates_STEPS,STEPS_Session_Start_Time,STEPS_Session_End_Time,STEPS_Hours_Range,Total_Dates_ACTIVE_ENERGY_BURNED,ACTIVE_ENERGY_BURNED_Session_Start_Time,ACTIVE_ENERGY_BURNED_Session_End_Time,ACTIVE_ENERGY_BURNED_Hours_Range,submission_index
0,0591b980-3942-4592-87a5-8aa274265c09,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,healthdata,1NPPILOT,21,all,1.0.0 - QC Pro (49),CET,2021-02-22 16:44:42.030916,2021-02-22T00:00:00.000,...,20210222,20210222-8146.0_,2021-02-22T08:39:59.008000,2021-02-22T17:30:13.211000,8.84,NaN,NaN,NaN,NaN,1
1,d1675237-4cd4-4f79-8740-0027d675ba43,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,healthdata,1NPPILOT,24,all,1.0.0 - QC Pro (50),CET,2021-02-23 08:00:32.571227,2021-02-22T17:44:41.042,...,20210222,20210222-368.0_20210223-68.0_,2021-02-22T18:00:03.037000,2021-02-23T08:43:56.027000,14.73,NaN,NaN,NaN,NaN,2
2,139fe89b-224b-4245-bfe4-d7da05439f22,b0354b51-fedf-4445-b2e2-335141af6709,healthdata,1NPPILOT,24,all,1.0.0 - QC Pro (50),CET,2021-02-23 09:41:15.565456,2021-02-23T00:00:00.000,...,20210223,20210223-33.0_,2021-02-23T07:02:49.014000,2021-02-23T07:03:45.177000,0.02,NaN,NaN,NaN,NaN,1
3,a7980e18-3b6b-4ccc-bc4f-6cb4ea90bd1e,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,healthdata,1NPPILOT,24,all,1.0.0 - QC Pro (50),CET,2021-02-23 13:26:50.345672,2021-02-23T09:00:31.842270,...,20210223,20210223-8743.0_,2021-02-23T10:02:23.981000,2021-02-23T13:53:13.989000,3.85,NaN,NaN,NaN,NaN,3
4,fbff0ed8-cce1-4bb2-af4b-d34d55ae5453,a7a7d332-c023-41dc-a36b-3b59e7f2deaa,healthdata,1NPPILOT,24,all,1.0.0 - QC Pro (54),CET,2021-02-23 21:24:23.690507,2021-02-23T00:00:00.000,...,20210223,20210223-138.0_,2021-02-23T10:09:00.832000,2021-02-23T21:56:20.309000,11.79,NaN,NaN,NaN,NaN,1


In [ ]:
# Dropping calories cols only because there is NO cal data
health_df = health_df.drop(['Total_Dates_ACTIVE_ENERGY_BURNED','ACTIVE_ENERGY_BURNED_Session_Start_Time',
'ACTIVE_ENERGY_BURNED_Session_End_Time','ACTIVE_ENERGY_BURNED_Hours_Range', 'Date_as_Number'], axis=1)

### Creating columns containing the amount of steps (Date_Steps_Count)  for each date from Total_Dates_STEPS

In [ ]:
unique_dates = []
for i, row in health_df.iterrows():
    step_counts = row["Total_Dates_STEPS"]
    for date_steps in step_counts.split("_"):
        if not date_steps:
            break
        pair_split = date_steps.split("-")
        date = pair_split[0] + '_Steps_Count'
        steps = pair_split[1]
        health_df.at[i, date] = steps

        if date not in unique_dates:
            unique_dates.append(date)

### Dropping, renaming columns and adding PPT_NR

In [ ]:
# # submission_index dropped
columns_to_drop_health = ['id','response_type', 'study', 'study_version', 'activity', 'received_at', 'time_start', 
                    'app_version', 'timezone', 'time_end', 'time_scheduled_start', 'time_scheduled_end','submission_index']

health_df_clean = health_df.drop(columns_to_drop_health, axis=1)

id_to_number(health_df_clean)

health_df_clean.rename({'participant':'Participant_ID'}, axis=1, inplace=True)

cols = health_df_clean.columns.tolist()
cols = [cols[-1]] + cols[0:-1]
health_df_clean = health_df_clean[cols]

health_df_clean.sort_values(by=['Participant_NR'], inplace=True)
health_df_clean.head()

,Participant_NR,Participant_ID,submitted_at,Total_Dates_STEPS,STEPS_Session_Start_Time,STEPS_Session_End_Time,STEPS_Hours_Range,20210222_Steps_Count,20210223_Steps_Count,20210224_Steps_Count,...,20210404_Steps_Count,20210405_Steps_Count,20210406_Steps_Count,20210407_Steps_Count,20210408_Steps_Count,20210409_Steps_Count,20210410_Steps_Count,20210411_Steps_Count,20210412_Steps_Count,20210413_Steps_Count
0,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,2021-02-22T17:44:41.081699,20210222-8146.0_,2021-02-22T08:39:59.008000,2021-02-22T17:30:13.211000,8.84,8146.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,2021-02-26T15:52:25.698078,20210224-9465.0_20210225-5211.0_20210226-10287.0_,2021-02-24T07:37:07.036000,2021-02-26T15:25:38.315000,55.81,NaN,NaN,9465.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,2021-02-27T09:55:44.287959,20210226-1442.0_20210227-199.0_,2021-02-26T16:17:32.909000,2021-02-27T09:36:50.939000,17.32,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,2021-03-13T12:49:24.609274,20210311-10200.0_20210312-2062.0_20210313-746.0_,2021-03-11T13:31:43.493000,2021-03-13T12:39:43.448000,47.13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,2021-02-27T14:19:03.084519,20210227-2472.0_,2021-02-27T09:59:55.788000,2021-02-27T13:49:04.482000,3.82,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
health_df_clean.tail(30)

,Participant_NR,Participant_ID,submitted_at,Total_Dates_STEPS,STEPS_Session_Start_Time,STEPS_Session_End_Time,STEPS_Hours_Range,20210222_Steps_Count,20210223_Steps_Count,20210224_Steps_Count,...,20210404_Steps_Count,20210405_Steps_Count,20210406_Steps_Count,20210407_Steps_Count,20210408_Steps_Count,20210409_Steps_Count,20210410_Steps_Count,20210411_Steps_Count,20210412_Steps_Count,20210413_Steps_Count
241,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-31T19:06:32.121921,20210330-2336.0_20210331-4727.0_,2021-03-30T18:21:10.458000,2021-03-31T18:23:33.588000,24.04,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-18T17:47:26.026402,20210318-2604.0_,2021-03-18T08:15:54.577000,2021-03-18T17:25:59.935000,9.17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-27T11:53:15.990740,20210327-83.0_,2021-03-27T11:16:40.283000,2021-03-27T11:20:56.857000,0.07,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-20T21:48:12.378831,20210320-113.0_,2021-03-20T00:05:04.738000,2021-03-20T10:40:47.128000,10.60,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-28T09:22:33.521488,20210327-14463.0_,2021-03-27T12:13:54.034000,2021-03-27T23:38:10.145000,11.40,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-25T17:36:37.279965,20210325-2831.0_,2021-03-25T08:17:51.628000,2021-03-25T17:24:34.299000,9.11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-19T18:34:57.825264,20210319-2008.0_,2021-03-19T08:45:41.790000,2021-03-19T18:19:12.106000,9.56,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-27T00:31:54.154312,20210326-1101.0_20210327-49.0_,2021-03-26T18:17:05.509000,2021-03-27T00:10:51.414000,5.90,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-29T08:21:34.912838,20210328-5796.0_20210329-1351.0_,2021-03-28T12:40:28.082000,2021-03-29T07:50:54.804000,19.17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,15.0,b915dd7f-59ef-4a13-a3b7-d2469b1d2605,2021-03-22T08:19:38.270704,20210320-1399.0_20210321-5685.0_20210322-2797.0_,2021-03-20T23:33:03.906000,2021-03-22T07:51:16.339000,32.30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Health Data: melt function

In [ ]:
columns_to_keep = ['Participant_NR', 'Participant_ID', 'STEPS_Session_Start_Time', 'STEPS_Session_End_Time','STEPS_Hours_Range']

health_df_melt = pd.melt(health_df_clean, id_vars=columns_to_keep, value_vars=unique_dates, value_name='Steps')
health_df_melt.sort_values(by=['variable'], inplace=True)
health_df_melt.head(31)

,Participant_NR,Participant_ID,STEPS_Session_Start_Time,STEPS_Session_End_Time,STEPS_Hours_Range,variable,Steps
0,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,2021-02-22T08:39:59.008000,2021-02-22T17:30:13.211000,8.84,20210222_Steps_Count,8146.0
175,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-02-24T07:39:45.177000,2021-02-24T07:48:45.330000,0.15,20210222_Steps_Count,NaN
176,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-04-03T13:54:34.972000,2021-04-04T10:24:46.015000,20.50,20210222_Steps_Count,NaN
177,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-04-02T17:05:34.088000,2021-04-03T12:50:25.901000,19.75,20210222_Steps_Count,NaN
178,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-03-25T21:14:33.589000,2021-03-26T07:22:25.348000,10.13,20210222_Steps_Count,NaN
179,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-03-26T21:31:58.643000,2021-03-26T21:45:48.172000,0.23,20210222_Steps_Count,NaN
180,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-03-27T10:07:35.844000,2021-03-27T16:49:07.473000,6.69,20210222_Steps_Count,NaN
181,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-04-01T13:27:52.531000,2021-04-02T08:06:21.430000,18.64,20210222_Steps_Count,NaN
182,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-03-27T17:29:58.187000,2021-03-28T09:56:47.623000,16.45,20210222_Steps_Count,NaN
183,12.0,506c4dd8-d2d3-4007-af82-5bed70681635,2021-03-31T22:06:01.239000,2021-04-01T12:48:53.380000,14.71,20210222_Steps_Count,NaN


In [ ]:
health_df_melt.rename({'STEPS_Hours_Range': 'Session_Duration_(hours)',
                            'variable': 'Date_as_Number'}, axis=1, inplace=True)


for i, row in health_df_melt.iterrows(): 
    health_df_melt.at[i, 'Date_as_Number'] = int(row['Date_as_Number'].split('_')[0])


health_df_to_merge = health_df_melt[health_df_melt['Steps'].notnull()]

cols = health_df_to_merge.columns.tolist()
cols = cols[0:2] + [cols[-2]] + [cols[-1]] + cols[2:5]
health_df_to_merge = health_df_to_merge[cols]

health_df_to_merge.sort_values(by=['Participant_NR','Date_as_Number']).head(31)

,Participant_NR,Participant_ID,Date_as_Number,Steps,STEPS_Session_Start_Time,STEPS_Session_End_Time,Session_Duration_(hours)
0,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210222,8146.0,2021-02-22T08:39:59.008000,2021-02-22T17:30:13.211000,8.84
17,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210222,368.0,2021-02-22T18:00:03.037000,2021-02-23T08:43:56.027000,14.73
293,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210223,68.0,2021-02-22T18:00:03.037000,2021-02-23T08:43:56.027000,14.73
299,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210223,5610.0,2021-02-23T16:31:49.331000,2021-02-24T06:54:32.914000,14.38
295,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210223,8743.0,2021-02-23T10:02:23.981000,2021-02-23T13:53:13.989000,3.85
553,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210224,9465.0,2021-02-24T07:37:07.036000,2021-02-26T15:25:38.315000,55.81
575,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210224,138.0,2021-02-23T16:31:49.331000,2021-02-24T06:54:32.914000,14.38
829,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210225,5211.0,2021-02-24T07:37:07.036000,2021-02-26T15:25:38.315000,55.81
1105,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210226,10287.0,2021-02-24T07:37:07.036000,2021-02-26T15:25:38.315000,55.81
1106,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210226,1442.0,2021-02-26T16:17:32.909000,2021-02-27T09:36:50.939000,17.32


In [ ]:
health_df_to_merge['Steps'] = health_df_to_merge['Steps'].astype(float)
health_df_grouped = health_df_to_merge.groupby(['Participant_NR', 'Participant_ID', 'Date_as_Number']).agg({'Steps': "sum"}).reset_index()
health_df_grouped 


,Participant_NR,Participant_ID,Date_as_Number,Steps
0,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210222,8514.0
1,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210223,14421.0
2,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210224,9603.0
3,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210225,5211.0
4,1.0,aef380fa-2be8-43c1-a17f-fe4b47d6a89f,20210226,11729.0
...,...,...,...,...
415,19.0,bfe27b74-5c43-44dc-b16b-fb7a039d1d65,20210306,7396.0
416,19.0,bfe27b74-5c43-44dc-b16b-fb7a039d1d65,20210307,31438.0
417,19.0,bfe27b74-5c43-44dc-b16b-fb7a039d1d65,20210308,5664.0
418,19.0,bfe27b74-5c43-44dc-b16b-fb7a039d1d65,20210309,523.0


In [ ]:
health_df_grouped.groupby(['Participant_NR', 'Date_as_Number']).count()['Participant_ID'].value_counts()

1    420
Name: Participant_ID, dtype: int64

In [ ]:
intake_clean.to_csv('intake_clean.csv', index= False)
gad_clean.to_csv('gad7_clean.csv', index= False)
phq_clean.to_csv('phq9_clean.csv', index= False)
panas_clean.to_csv('panas10_clean.csv', index= False)
tapping_clean.to_csv('tapping_clean.csv', index= False)
stroop_clean.to_csv('stroop_clean.csv', index= False)
stroop_basic.to_csv('stroop_basic.csv', index= False)
health_df_to_merge.to_csv('health_data_to_merge_with_cal.csv', index=False)
health_df_grouped.to_csv('health_data_per_day.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b4078eea-bcf6-4844-b765-c2c73e45d4d9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>